In [1]:
!pip install rdkit
!pip install mordred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 28.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.0 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=b2c56731474bab18faf57b4bb8196a09816352d6401ccd9fc15c12de0f7569a5
  Stored in directory: /root/.cache/pip/wheels/05/95/d1/9e913738f0e8362b3676917b953a60afd76d2b0b99ff8a71ec
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [2]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import rdmolfiles, rdmolops, AllChem, Draw, rdDepictor, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Fingerprints import FingerprintMols

from IPython.display import SVG
from IPython.display import HTML
from IPython.display import display

from collections import defaultdict
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('./drive/MyDrive/AI/Material Informatics (MI)/Ring Repeating Units/Table of Supporting Information(SMILES)- Cal band gap_(RU1, RU2, RRU).csv')

In [5]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930


In [6]:
df["mol_ru1"] = df["SMILES_RU1"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_ru2"] = df["SMILES_RU2"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_rru"] = df["SMILES_RRU"].apply(lambda x:Chem.MolFromSmiles(x))

In [7]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV,mol_ru1,mol_ru2,mol_rru
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691,<rdkit.Chem.rdchem.Mol object at 0x7f0411bdea50>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b8d7b0>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b995f0>
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041,<rdkit.Chem.rdchem.Mol object at 0x7f0411bde970>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b8d890>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b996d0>
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807,<rdkit.Chem.rdchem.Mol object at 0x7f0411bde900>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b8d820>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b99660>
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918,<rdkit.Chem.rdchem.Mol object at 0x7f0411bde9e0>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b8d900>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b99740>
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930,<rdkit.Chem.rdchem.Mol object at 0x7f0411bdeac0>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b8d970>,<rdkit.Chem.rdchem.Mol object at 0x7f0411b997b0>


In [8]:
#MACCS Keys Fingerprints
def maccskeys_fingerprint(mols):
  maccs_fps = [AllChem.GetMACCSKeysFingerprint(mol) for mol in mols]
  df_maccs = pd.DataFrame(np.array(maccs_fps, int))
  return df_maccs

In [9]:
maccs_ru1 = maccskeys_fingerprint(df["mol_ru1"])
maccs_ru1 = maccs_ru1.dropna(1)

maccs_ru2 = maccskeys_fingerprint(df["mol_ru2"])
maccs_ru2 = maccs_ru2.dropna(1)

maccs_rru = maccskeys_fingerprint(df["mol_rru"])
maccs_rru = maccs_rru.dropna(1)

<ipython-input-9-a5c63558547e>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  maccs_ru1 = maccs_ru1.dropna(1)
<ipython-input-9-a5c63558547e>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  maccs_ru2 = maccs_ru2.dropna(1)
<ipython-input-9-a5c63558547e>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  maccs_rru = maccs_rru.dropna(1)


In [10]:
# A list for the evaluation result
rmse_sum = {}
r2_sum = {}

RU1

In [11]:
from sklearn.model_selection import train_test_split

X = maccs_ru1.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [12]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [13]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru1: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru1: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru1: 0.5633454333456847
RMSE(Train): 0.8043055976497916

R_square(Test)_ru1: 0.6323387947169137
RMSE(Test): 0.637335790327849


In [14]:
rmse_sum['RU1'] = [RMSE_train, RMSE_test] 
r2_sum['RU1'] = [R_square_train, R_square_test] 

RU2

In [15]:
from sklearn.model_selection import train_test_split

X = maccs_ru2.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [16]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [17]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru2: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru2: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru2: 0.5659151966023239
RMSE(Train): 0.8019353889521669

R_square(Test)_ru2: 0.6322795284407294
RMSE(Test): 0.6373871569071037


In [18]:
rmse_sum['RU2'] = [RMSE_train, RMSE_test] 
r2_sum['RU2'] = [R_square_train, R_square_test] 

RRU

In [19]:
from sklearn.model_selection import train_test_split

X = maccs_rru.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [20]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [21]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_rru: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_rru: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_rru: 0.5836959604706249
RMSE(Train): 0.7853394256765852

R_square(Test)_rru: 0.6661324770042879
RMSE(Test): 0.6073394498350366


In [22]:
rmse_sum['RRU'] = [RMSE_train, RMSE_test] 
r2_sum['RRU'] = [R_square_train, R_square_test] 

Summary

In [23]:
summary = pd.DataFrame(r2_sum, index=["train", "test"])
summary.style.set_caption("Table of R Square (Maccs keys)")

,RU1,RU2,RRU
train,0.563345,0.565915,0.583696
test,0.632339,0.632280,0.666132


In [25]:
summary = pd.DataFrame(rmse_sum, index=["train", "test"])
summary.style.set_caption("Table of RMSE (Maccs keys)")

,RU1,RU2,RRU
train,0.804306,0.801935,0.785339
test,0.637336,0.637387,0.607339
